In [3]:
import os
import sys
sys.path.append('../part12')
import random
import numpy as np

from cv23_lab1_part3_utils import featuresSURF
from cv23_lab1_part3_utils import featuresHOG
from cv23_lab1_part3_utils import matching_evaluation
from cv23_lab1_part3_utils import FeatureExtraction
from cv23_lab1_part3_utils import createTrainTest
from cv23_lab1_part3_utils import BagOfWords
from cv23_lab1_part3_utils import svm
from corner_detection_utils import CornerDetection
from corner_detection_utils import HarrisLaplacian
from blob_detection_utils import BlobDetection
from blob_detection_utils import HessianLaplacian
from box_detection_utils import BoxLaplacian

# Image Matching

In [4]:
rotations = [-20, -10, 0, 10, 20]
escalations = [0.6, 0.8, 1.0, 1.2]
transformations = [(r, e) for r in rotations for e in escalations]

In [5]:
descriptors = [lambda I, kp: featuresSURF(I, kp), lambda I, kp: featuresHOG(I, kp)]
detectors = [lambda i: CornerDetection(i, 2, 2.5, 0.005, 0.05),
             lambda i: HarrisLaplacian(i, 2, 2.5, 0.005, 0.05, 1.1, 6),
             lambda i: BlobDetection(i, 2, 0.005),
             lambda i: HessianLaplacian(i, 2, 0.005, 1.1, 6),
             lambda i: BoxLaplacian(i, 2, 0.005, 1.1, 6)]
descriptor_names = ["featuresSURF", "featuresHOG"]
detector_names = ["CornerDetection", "HarrisLaplacian", "BlobDetection", "HessianLaplacian", "BoxLaplacian"]
detector_descriptor = [(detector, descriptor)
                       for detector in detectors
                       for descriptor in descriptors]
detector_descriptor_names = [(detector, descriptor)
                             for detector in detector_names
                             for descriptor in descriptor_names]

In [25]:
# this takes around 1-2 minutes
avg_scale_errors_list = np.zeros((len(descriptors), len(detectors), 3))
avg_theta_errors_list = np.zeros((len(descriptors), len(detectors), 3))
for index, descriptor in enumerate(descriptors):
    for jndex, detector in enumerate(detectors):
        avg_scale_errors_list[index, jndex], avg_theta_errors_list[index, jndex] = matching_evaluation(detector, descriptor)

In [44]:
np.set_printoptions(precision=3)
minim = np.mean(avg_scale_errors_list[0,0])
for index, descriptor in enumerate(descriptor_names):
    for jndex, detector in enumerate(detector_names):
        if minim > np.mean(avg_scale_errors_list[index, jndex]):
            minim = np.mean(avg_scale_errors_list[index, jndex])
            bestpair = (descriptor, detector)
with open("../report/avg_scale_errors.txt", 'w') as file:
    for index, descriptor in enumerate(descriptor_names):
        for jndex, detector in enumerate(detector_names):
            if (descriptor, detector) == bestpair:
                file.write(f"{descriptor}, {detector} :\t{avg_scale_errors_list[index, jndex]}\t BEST\n")
            else:
                file.write(f"{descriptor}, {detector} :\t{avg_scale_errors_list[index, jndex]}\t\n")

minim = np.mean(avg_theta_errors_list[0,0])
for index, descriptor in enumerate(descriptor_names):
    for jndex, detector in enumerate(detector_names):
        if minim > np.mean(avg_scale_errors_list[index, jndex]):
            minim = np.mean(avg_scale_errors_list[index, jndex])
            bestpair = (descriptor, detector)
with open("../report/avg_theta_errors.txt", 'w') as file:
    for index, descriptor in enumerate(descriptor_names):
        for jndex, detector in enumerate(detector_names):
            if (descriptor, detector) == bestpair:
                file.write(f"{descriptor}, {detector} :\t{avg_theta_errors_list[index, jndex]}\t BEST\n")
            else:
                file.write(f"{descriptor}, {detector} :\t{avg_theta_errors_list[index, jndex]}\n")


# Feature Extraction

In [6]:
files = ["./features-trial1/"+detector+'_'+descriptor+".txt"
               for detector in detector_names
               for descriptor in descriptor_names]
for name in files:
    os.mknod(name)
functions_and_files = list(zip(detector_descriptor, files))

In [7]:
# this takes around 20 minutes
# DO NOT EXECUTE THIS CELL IF THE "FEATURES" DIRECTORY EXISTS
# IF IT EXISTS, EXECUTE THE NEXT CELL.
features_list = [FeatureExtraction(detector, descriptor, saveFile=store)
                 for ((detector, descriptor), store) in functions_and_files]

Time for feature extraction: 13.159
Time for feature extraction: 18.446
Time for feature extraction: 123.690
Time for feature extraction: 111.589
Time for feature extraction: 11.076
Time for feature extraction: 23.881
Time for feature extraction: 96.706
Time for feature extraction: 96.918
Time for feature extraction: 111.353
Time for feature extraction: 102.755


In [7]:
# this is instant provided that the files exist
features_list = [FeatureExtraction(detector, descriptor, loadFile=store) 
                   for ((detector, descriptor), store) in functions_and_files]

In [10]:
results = []
for index, feats in enumerate(features_list):
    accs = []
    for k in range(5):
        # Split into a training set and a test set.
        data_train, label_train, data_test, label_test = createTrainTest(feats, k)

        # Perform Kmeans to find centroids for clusters.
        BOF_tr, BOF_ts = BagOfWords(data_train, data_test)
        # print(BOF_tr.shape, BOF_ts.shape)

        # Train an svm on the training set and make predictions on the test set
        acc, preds, probas = svm(BOF_tr, label_train, BOF_ts, label_test)
        accs.append(acc)
    print(f"{100.0*np.mean(accs):.3f}")
    detector, descriptor = detector_descriptor_names[index]
    results.append(f"{detector}, {descriptor} :\t{100.0*np.mean(accs):.3f}\n")
    #with open(f"mean_accuracies/mean_acc_{detector}_{descriptor}.txt", 'w') as file:
    #    file.write(f"{100.0*np.mean(accs):.3f}")
with open("mean_acc_bovw_trial1_results_test.txt", 'w') as file:
    for result in results:
        file.write(result)

53.793
62.069
59.862
65.517
56.138
65.655
57.517
66.483
51.586
61.517


In [6]:
def kmeans(X, k, max_iters=100):
    """
    K-means clustering algorithm.
    
    Parameters
    ----------
    X : numpy.ndarray
        The input data points, of shape (n_samples, n_features).
    k : int
        The number of clusters to form.
    max_iters : int, optional
        The maximum number of iterations to run the algorithm.
    
    Returns
    -------
    centroids : numpy.ndarray
        The final cluster centroids, of shape (k, n_features).
    labels : numpy.ndarray
        The index of the cluster each data point belongs to, of shape (n_samples,).
    """
    
    n_samples, _ = X.shape
    
    # Initialize the cluster centroids randomly
    centroids = X[np.random.choice(n_samples, size=k, replace=False)]
    
    for _ in range(max_iters):
        # Assign each data point to the nearest cluster centroid
        distances = np.linalg.norm(X[:, np.newaxis, :] - centroids, axis=-1) # shape (n_samples, k)
        labels = np.argmin(distances, axis=-1) # shape (n_samples,)
        
        # Update the cluster centroids as the mean of the data points in each cluster
        for i in range(k):
            mask = (labels == i)
            if np.sum(mask) > 0:
                centroids[i] = np.mean(X[mask], axis=0)
    
    return centroids, labels

In [17]:
# this is extremely slow
def myBagOfVisualWords(train, test):
    train_feature = np.concatenate(train, axis=0)
    test_feature = np.concatenate(test, axis=0)
    
    length = len(train_feature)
    amount = int(length*0.5)
    random_elements_train = train_feature[np.random.choice(length, size=amount, replace=False)]
    clusters = random.randint(500,2000)
    centroids, _ = kmeans(random_elements_train, clusters)
    k, _ = centroids.shape

    # Function to compute the BoVW representation for a set of images
    def compute_bovw(feature_set):
        bovw_set = []
        for features in feature_set:
            distances = np.linalg.norm(features[:, np.newaxis, :] - centroids, axis=-1)
            labels = np.argmin(distances, axis=-1)
            counts = np.bincount(labels, minlength=k)
            l2norm = np.sqrt(np.sum(counts * counts))
            bovw = counts / l2norm
            bovw_set.append(bovw)
        return np.array(bovw_set)

    # Compute the BoVW representation for the training and testing sets
    train_bovw = compute_bovw(train)
    test_bovw = compute_bovw(test)

    return train_bovw, test_bovw

In [23]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans

def myFastBagOfVisualWords(train, test, clusters=500, maxiter=100):
    train_feature = np.concatenate(train, axis=0)
    #test_feature = np.concatenate(test, axis=0)

    # Set the number of clusters (visual words)
    # you can choose random number of clusters
    # clusters = random.randint(500, 2000)
    # clusters = 1000  # You can change this value

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=clusters, random_state=0, max_iter=maxiter, tol=0.001).fit(train_feature)

    # Function to compute the BoVW representation for a set of images
    def compute_bovw(feature_set):
        bovw_set = []
        for features in feature_set:
            distances = cdist(features, kmeans.cluster_centers_, metric='euclidean')
            labels = np.argmin(distances, axis=-1)
            counts = np.bincount(labels, minlength=clusters)
            l2norm = np.sqrt(np.sum(counts * counts))
            bovw = counts / l2norm
            bovw_set.append(bovw)
        return np.array(bovw_set)

    # Compute the BoVW representation for the training and testing sets
    train_bovw = compute_bovw(train)
    test_bovw = compute_bovw(test)

    return train_bovw, test_bovw


In [ ]:
# this takes close to an hour
# perhaps the precompiled version uses less clusters and 
# less max_iterations? I don't know.
for index, feats in enumerate(features_list):
    accs = []
    for k in range(5):
        # Split into a testing set and a test set.
        data_train, label_train, data_test, label_test = createTrainTest(feats, k)

        # Perform Kmeans to find centroids for clusters.
        BOF_tr, BOF_ts = myFastBagOfVisualWords(data_train, data_test, clusters=1000, maxiter=300)
        # Perform Kmeans to find centroids for clusters.
        #BOF_tr, BOF_ts = BagOfWords(data_train, data_test)
        print(BOF_tr.shape, BOF_ts.shape)

        # Train an svm on the training set and make predictions on the test set
        acc, preds, probas = svm(BOF_tr, label_train, BOF_ts, label_test)
        accs.append(acc)
    detector, descriptor = detector_descriptor_names[index]
    results.append(f"{detector}, {descriptor} :\t{100.0*np.mean(accs):.3f}\n")
    #with open(f"mean_accuracies/mean_acc_{detector}_{descriptor}.txt", 'w') as file:
    #    file.write(f"{100.0*np.mean(accs):.3f}")
with open("mean_acc_mybovw_1000_trial1_results.txt", 'w') as file:
    for result in results:
        file.write(result)

In [24]:
# This is perhaps a little bit faster. It takes
# around 30 minutes instead of 60. The number of iterations
# do not seem to make it much better. Increasing
# the tolerance makes it a bit faster, but the
# accuracy is also slightly decreased.
for index, feats in enumerate(features_list):
    accs = []
    for k in range(5):
        # Split into a testing set and a test set.
        data_train, label_train, data_test, label_test = createTrainTest(feats, k)

        # Perform Kmeans to find centroids for clusters.
        BOF_tr, BOF_ts = myFastBagOfVisualWords(data_train, data_test, clusters=500, maxiter=100)
        # Perform Kmeans to find centroids for clusters.
        #BOF_tr, BOF_ts = BagOfWords(data_train, data_test)
        #print(BOF_tr.shape, BOF_ts.shape)

        # Train an svm on the training set and make predictions on the test set
        acc, preds, probas = svm(BOF_tr, label_train, BOF_ts, label_test)
        accs.append(acc)
    detector, descriptor = detector_descriptor_names[index]
    results.append(f"{detector}, {descriptor} :\t{100.0*np.mean(accs):.3f}\n")
    #with open(f"mean_accuracies/mean_acc_{detector}_{descriptor}.txt", 'w') as file:
    #    file.write(f"{100.0*np.mean(accs):.3f}")
with open("mean_acc_mybovw_500_trial1_results.txt", 'w') as file:
    for result in results:
        file.write(result)

(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500) (145, 500)
(337, 500)